In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# DataFrame from github locate_stat
df = pd.read_csv("C:/py_projects/stats_hotel/data_files/main", header=[0,1], index_col=0)

In [8]:
df.loc['2023-09-01'].profit
# month.loc['2022-08-01']

Показатель
2_rooms     32718.0
3_rooms     19601.6
1_room      33916.0
P2_rooms    33440.0
Name: 2023-09-01, dtype: float64

In [ ]:
# here is a path to file from travelline.ru(stats and reports)
month_var = input('type filename from data_files folder without extansion: ')
# creating DataFrame from this file
month = pd.read_excel(f'C:/py_projects/stats_hotel/data_files/{month_var}.xlsx', index_col='Показатель')

In [ ]:
month.index = month.index.str.strip()

In [ ]:
month

In [ ]:
rows_to_exclude = ["Мансарда. Стандарт с 2 раздельными кроватями", "Мансарда. Улучшенный номер 2-х местный",
                   "Мансарда. Семейный 3-местный с балконом", "Доход по еде и напиткам (F&B)", "Доходы прочие",
                   "Заказано допуслуг на сумму", "Доход за период", "Завтрак", "Гость без места", "Завтрак континентальный",
                   "Штрафы", "Бесплатная парковка", "Аренда лежаков, Бассейн, Уборка", "Pechory Apartment (Gagarina 1 room)"]

In [ ]:
month = month[~month.index.isin(rows_to_exclude)]

In [ ]:
month = month.rename(index={'Процент загрузки отеля': 'load_percent',
                            'Средняя стоимость номера за сутки (ADR)':'room_coast_mean',
                            'Средний доход на номер (RevPar)': 'room_profit_mean',
                            'Забронировано номеров/день': 'rooms_per_day',
                            '№1 Двухуровневый люкс, 1 линия, вид на море': 'N1',
                            '№2 Стандартный номер, 1 линия, боковой вид на море (3чел)': 'N2',
                            '№3 Полулюкс улучшенный, 1 линия, вид на море': 'N3',
                            '№4 Стандарт, 1 линия, боковой вид на море (4чел)': 'N4',
                            'Доход за проживание': 'profit',
                            'Количество гостей': 'guests',
                            '№5 Полулюкс, 2 линия, вид на сад': 'N5',
                            "№6 Стандарт, 2 линия, вид на сад": 'N6',
                            '№7 Двухуровневый люкс, 2 линия, боковой вид на море': 'N7',
                            '№8 Стандарт улучшенный, 2 линия, боковой вид на море': 'N8',
                            '2-х комнатные Апартаменты с 1 спальней': '2_rooms',
                            '3-х комнатные апартаменты с 2-я спальнями': '3_rooms',
                            'Апартаменты с 1-й спальней': '1_room',
                            '2-х комнатные апартаменты': 'P2_rooms'})

In [ ]:
month = month.rename(columns={'Значение за период': '2023-03-01'})

In [ ]:
for c1 in month.columns:
    v = month[c1]
    month[c1] = v.str.replace(",", ".", regex=False)

In [ ]:
for c1 in month.columns:
    v = month[c1]
    month[c1] = v.str.replace("[\s, %, R, U, B]", "", regex=True).astype(float)

In [ ]:
# This function evaluating DF index to Multindex(like main DF)
def group_rows(x):
    name = x.index[0]
    x = x.iloc[1:]
    x.index = pd.MultiIndex.from_product([[name], x.index])
    return x

In [ ]:
month.shape

In [ ]:
month = month.groupby(np.arange(len(month)) // 5, group_keys=False).apply(group_rows).T

In [ ]:
month.columns

Дальше необходимо подумать над необходимостью удаления входящих файлов в папке

In [ ]:
df.loc['2023-03-01'] = month.loc['2023-03-01']

In [ ]:
frames = [df, month]
result = pd.concat(frames)

In [ ]:
result

In [ ]:
df.to_csv('C:/py_projects/stats_hotel/data_files/main')
# os.remove(f'C:/py_projects/stats_hotel/data_files/{month_var}.xlsx')

In [ ]:
# just check-up all work before 
print(f'main is extended by {month_var}')
print('initial scale: ', df.shape)
print('finite scale: ', result.shape)
print(result.index)